
## Downloading Multiple files

### Often, it is possible to work out from a few files that there is a pattern in the way files and hence the URLs are created

### In this example we will make use of such a pattern to allow us to download multiple files

### The libraries we need

In [ ]:
import requests
import pandas as pd
import os

## The code below is similar to what we have seen before

### The file comes from the Webpage `https://www.getthedata.com/covid-19/utla-by-day`

### With the link to the download file shown as `cases_by_utla_2020-03-16.csv`
### This is infact a relative address from the website.

## But the FULL url of the actual file we would download is shown at the bottom left of the screen as we hover over the link. It is:

##  ![Actual Filename](./images/actual_filename.jpg)

## If we want to download the file by other means then this is the URL that we must use.

In [ ]:

myurl = 'https://www.getthedata.com/downloads/cases_by_utla_2020-03-16.csv'

savefilename = '200316.csv'

r = requests.get(myurl)
print(r.status_code)
file = open(savefilename, "wb")
file.write(r.content)
file.close()

## If we wanted to download multiple files we can use the structure of the filenames and requests the downloads in a loop structure
### Like we see below. 
### As we want to save the files we use a similar technique to name the output files.

In [ ]:
stem = 'https://www.getthedata.com/downloads/cases_by_utla_2020-03-'
ftype = '.csv'

ym = 'd:/covid19'
date_stem = '2020-03-'

for i in range(16,32):
    print(f'{stem}{i}{ftype}')
    print(f'{ym}/{date_stem}{i}{ftype}')    

## Then we can put it all together

In [ ]:
stem = 'https://www.getthedata.com/downloads/cases_by_utla_2020-03-'
ftype = '.csv'

ym = 'd:/covid19'
date_stem = '2020-03-'

for i in range(16,32):
    # set up URL and filename
    myurl = stem + str(i) + ftype
    savefilename = ym + "/" + date_stem + str(i) + ftype
    
    # Make the request
    r = requests.get(myurl)
    print(r.status_code)
    
    # Write the output file
    file = open(savefilename, "wb")
    file.write(r.content)
    file.close()


## Exercise

### When will the above code run to problems?

### There are two problems both related to how the we iterate through the dates.

### We only downloaded the files to the end of March. If we wanted to go into April, the `date_stem` would need changing but also the the valur of `str(i)` for the first nine days would not have the required leading zero.

### We can resolve both of these problems at the same time using a datetime method in pandas.


In [ ]:
import pandas as pd

# Getting a list of dates

sdate = '2020-03-16'
edate = '2020-12-31'

date_list = pd.date_range(sdate, edate, freq='d')

print(date_list)
#

print(f'{date_list[0]}')
#

date_list[0].strftime('%Y-%m-%d')

## Things I need to change

1. The `stem`
2. Create list of dates
3. Iterate through dates in `for` loop
4. change `myurl` to include the string format of the date
5. cchange `savefilename` to include the string format of the date

In [ ]:
stem = 'https://www.getthedata.com/downloads/cases_by_utla_'
ftype = '.csv'

ym = 'd:/covid19'

# Getting a list of dates

sdate = '2020-03-16'
edate = '2020-12-31'

date_list = pd.date_range(sdate, edate, freq='d')

for date in date_list:
    # set up URL and filename
    myurl = stem + date.strftime('%Y-%m-%d') + ftype
    savefilename = ym + "/" + date.strftime('%Y-%m-%d') + ftype
    
    # Make the request
    r = requests.get(myurl)
    print(r.status_code)
    
    # Write the output file
    file = open(savefilename, "wb")
    file.write(r.content)
    file.close()

## Once downloaded we can use Excel Data | Import from other sources | from folder  to combine the files into one.


## Or we could use pandas to combine the files

#### Steps

1. Geta list of the files in the folder
2. Use pandas to create a dataframe for each
3. Combine all of the dataframes
4. Write them out to a csv file.

In [ ]:
import os
import pandas as pd

In [ ]:
## Get list of files

myfolder = 'd:' + os.sep +'covid19'


files = os.listdir(myfolder)

for f in files:
    print(f'{myfolder}{os.sep}{f}')


### But if we use these in a call to read_csv, it will fail

In [ ]:
df = pd.read_csv(files[0],encoding = "utf-8", delimiter = ",")

## We need filenames with the fullpath

In [ ]:
filename = myfolder + os.sep + files[0]
print(filename)

##  Now try it

In [ ]:
filename = myfolder + os.sep + files[0]
df = pd.read_csv(filename,encoding = "utf-8", delimiter = ",")
df.head()

In [ ]:
##  concatenate all into single pandas dataframe

In [ ]:
# Import first file to create a dataframe

filename = myfolder + os.sep + files[0]
df = pd.read_csv(filename,encoding = "utf-8", delimiter = ",")


# Then append all othe rest 
#
for file in files[1:len(files)]:
    filename = myfolder + os.sep + file
    df_csv = pd.read_csv(filename, encoding = "utf-8", delimiter = ",")
    df = df.append(df_csv)
    
df.shape

In [ ]:
df.head()

## The real problem is the lack of filename info 

### We can get the date from the filename


In [ ]:
print(files[0])


## And we can create a new column in the data frame

In [ ]:
df['Date'] = files[0][0:10]
df.head()

## So we end up with

In [ ]:
# Import first file to create a dataframe

filename = myfolder + os.sep + files[0]
df = pd.read_csv(filename,encoding = "utf-8", delimiter = ",")
df['Date'] = files[0][0:10]

# Then append all othe rest 
#
for file in files[1:len(files)]:
    filename = myfolder + os.sep + file
    df_csv = pd.read_csv(filename, encoding = "utf-8", delimiter = ",")
    df_csv['Date'] = file[0:10]
    df = df.append(df_csv)

In [ ]:
## Finally we write it out to a csv file

In [ ]:
df.to_csv('all_covid.csv', index = False)